In [2]:
#pip install atproto

## Se connecter au client

Doc : https://github.com/MarshalX/atproto

Charger mes identifiants

In [2]:
import yaml

with open("./../../creds.yaml", "r") as f: 
    creds = yaml.load(f, Loader=yaml.FullLoader)

In [3]:
from atproto import Client

client = Client()
client.login(creds["username"], creds["password"])

ProfileViewDetailed(did='did:plc:svofadxzzkfjjcjh3xzhy73v', handle='activetigger.bsky.social', associated=ProfileAssociated(chat=None, feedgens=0, labeler=False, lists=0, starter_packs=0, py_type='app.bsky.actor.defs#profileAssociated'), avatar='https://cdn.bsky.app/img/avatar/plain/did:plc:svofadxzzkfjjcjh3xzhy73v/bafkreieocqwf3shlt5uzpm6rf26z36utigo5ktwlqqppkvhkotu3yqmkxe@jpeg', banner=None, created_at='2025-03-09T16:02:10.241Z', description=None, display_name='', followers_count=1, follows_count=1, indexed_at='2025-03-09T16:02:10.241Z', joined_via_starter_pack=None, labels=[], pinned_post=None, posts_count=3, viewer=ViewerState(blocked_by=False, blocking=None, blocking_by_list=None, followed_by=None, following=None, known_followers=None, muted=False, muted_by_list=None, py_type='app.bsky.actor.defs#viewerState'), py_type='app.bsky.actor.defs#profileViewDetailed')

In [15]:
client.send_post("C'est le cours de CSS à Marseille")

CreateRecordResponse(uri='at://did:plc:svofadxzzkfjjcjh3xzhy73v/app.bsky.feed.post/3ll4t7irbkr26', cid='bafyreicqoezinqguw4vt3plglu2th7c7ldk6fgi4f7m62qilhwprdq2puy')

## Transformer une URL en URI (passer du web à Bluesky)

https://bsky.app/profile/benecologie.bsky.social/post/3ll4mgtcs7c2n

at://{did}/app.bsky.feed.post/{post_id}

In [17]:
response = client.resolve_handle("benecologie.bsky.social")
response.did

'did:plc:njq5tj6ear5urnuoicl2kukt'

In [19]:
key = "3ll4mgtcs7c2n"
did = "did:plc:njq5tj6ear5urnuoicl2kukt"
uri = f"at://{did}/app.bsky.feed.post/{key}"

In [20]:
uri

'at://did:plc:njq5tj6ear5urnuoicl2kukt/app.bsky.feed.post/3ll4mgtcs7c2n'

In [32]:
reponse = client.app.bsky.feed.get_posts({"uris":[uri]})

In [33]:
type(reponse)

atproto_client.models.app.bsky.feed.get_posts.Response

In [29]:
with open("post.json","w") as f:
    f.write(reponse.model_dump_json())

In [38]:
post = reponse.posts[0]

In [40]:
type(post)

atproto_client.models.app.bsky.feed.defs.PostView

In [6]:
def extract_info(post):
    """
    Function to extract information from a bluesky post object
    """
    return {
            "created_at":post.record.created_at,
            "text": post.record.text,
           "handle": post.author.handle,
           "displayname": post.author.display_name,
           "like":post.like_count,
           "repost":post.repost_count,
           "reply":post.reply_count}

In [54]:
extract_info(post)

{'text': "Il me semble que le mythe des hormones féminines dangereuses dans le soja transformé est un mensonge favorisé par l'industrie de le viande non ? \n\nQuelqu'un a plus d'infos ? (Merci 💚)",
 'handle': 'benecologie.bsky.social',
 'displayname': 'Une femme-soja',
 'like': 73,
 'repost': 11,
 'reply': 22}

## Chercher plus de posts !

In [57]:
response = client.app.bsky.feed.search_posts({"q":"llm ", "limit":100})

In [8]:
import pandas as pd

In [66]:
#pd.DataFrame([extract_info(post) for post in response.posts])

In [4]:
all_posts = []
cursor = None
n_elements = 10000
while True: 
    response = client.app.bsky.feed.search_posts({"q":"llm", 
                                                  "limit":100, 
                                                  "sort": 'latest',
                                                  "cursor":cursor, 
                                                 "lang":"fr"})
    
    all_posts.extend(response["posts"])
    cursor = response.cursor
    if not cursor:
        break  # No more posts to fetch
    if len(all_posts) >= n_elements:
        break
    print(len(all_posts))

99
199
299
399
499
599
699
799
890
990
1090
1190
1289
1389
1487
1586
1686
1786
1885
1984
2084
2184
2284
2384


In [5]:
len(all_posts)

2399

In [9]:
df = pd.DataFrame([extract_info(post) for post in all_posts])

In [11]:
df.to_csv("../../data_llm_bs.csv")

In [80]:
df.sort_values("like", ascending=False).iloc[1]["text"]

'Apple just paused its AI LLM Siri upgrade indefinitely. \n\nThe writing is on the wall.'